In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from src.utils import *
from src.cleaning import *

In [7]:
train = pd.read_pickle("meetup_data.pkl")
test = pd.read_pickle("meetup_test.pkl")

In [8]:
train.columns

Index(['Order No', 'User Id', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Pickup Lat', 'Pickup Long', 'Destination Lat', 'Destination Long',
       'Rider Id', 'Time from Pickup to Arrival', 'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'geopy_distance', 'Pickup - hour'],
      dtype='object')

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    train.drop(["Time from Pickup to Arrival"],axis=1), train['Time from Pickup to Arrival'], test_size=0.33, random_state=42)

In [13]:
import lightgbm as lgb
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_test, label=y_test)
gbm = lgb.train(params,
                d_train,
                num_boost_round=200,
                valid_sets=d_valid,
                early_stopping_rounds=10)


[1]	valid_0's l1: 737.647	valid_0's l2: 954132
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l1: 722.637	valid_0's l2: 922825
[3]	valid_0's l1: 708.792	valid_0's l2: 894554
[4]	valid_0's l1: 695.594	valid_0's l2: 868041
[5]	valid_0's l1: 683.567	valid_0's l2: 844440
[6]	valid_0's l1: 672.288	valid_0's l2: 823092
[7]	valid_0's l1: 661.917	valid_0's l2: 803875
[8]	valid_0's l1: 652.347	valid_0's l2: 786442
[9]	valid_0's l1: 643.468	valid_0's l2: 770758
[10]	valid_0's l1: 635.253	valid_0's l2: 756197
[11]	valid_0's l1: 627.149	valid_0's l2: 741849
[12]	valid_0's l1: 619.952	valid_0's l2: 729162
[13]	valid_0's l1: 613.11	valid_0's l2: 717132
[14]	valid_0's l1: 606.704	valid_0's l2: 706528
[15]	valid_0's l1: 600.888	valid_0's l2: 697036
[16]	valid_0's l1: 595.791	valid_0's l2: 688809
[17]	valid_0's l1: 590.997	valid_0's l2: 681382
[18]	valid_0's l1: 586.296	valid_0's l2: 673764
[19]	valid_0's l1: 581.705	valid_0's l2: 666476
[20]	valid_0's l1: 577.398	valid_0's

In [14]:
df_sample_submission = pd.read_csv('data/SampleSubmission.csv')
df_variable_definitions = pd.read_csv('data/VariableDefinitions.csv')

In [15]:
create_submission(df_sample=df_sample_submission, df_test=test, model=gbm, name='trial')

,Time from Pickup to Arrival
Order_No,
Order_No_19248,1677.600405
Order_No_12736,2975.239718
Order_No_768,1926.355572
Order_No_15332,2480.497734
Order_No_21373,2644.217422
...,...
Order_No_3612,1489.892597
Order_No_7657,2941.013545
Order_No_1969,1779.309163
